In [6]:
from transformers import pipeline
from transformers.tokenization_utils import TruncationStrategy

import tokenizers
import pandas as pd
import requests



models = {
    "AlephBERT-base": {
        "name_or_path":"onlplab/alephbert-base",
        "description":"AlephBERT base model",
    },
    "HeBERT-base-TAU": {
        "name_or_path":"avichr/heBERT",
        "description":"HeBERT model created by TAU"
    },
    "mBERT-base-multilingual-cased": {
        "name_or_path":"bert-base-multilingual-cased",
        "description":"Multilingual BERT model"
    }
}

def get_json_from_url(url):
    return models
    return requests.get(url).json()

# models = get_json_from_url('https://huggingface.co/spaces/biu-nlp/AlephBERT/raw/main/models.json')



def load_model(model):
    pipe = pipeline('fill-mask', models[model]['name_or_path'])
    def do_tokenize(inputs):
        return pipe.tokenizer(
                inputs,
                add_special_tokens=True,
                return_tensors=pipe.framework,
                padding=True,
                truncation=TruncationStrategy.DO_NOT_TRUNCATE,
            )

    def _parse_and_tokenize(
        inputs, tokenized=False, **kwargs
    ):
        if not tokenized:
            inputs = do_tokenize(inputs)
        return inputs

    pipe._parse_and_tokenize = _parse_and_tokenize
    
    return pipe, do_tokenize






mode = 'Models'

if mode == 'Models':
    model = "AlephBERT-base"
    masking_level = 'Tokens'
    
    model_tags = model.split('-')
    model_tags[0] = 'Model:' + model_tags[0] 


    unmasker, tokenize = load_model(model)
    input_text = " [MASK] אתמול הלכתי "      
    input_masked = None
    tokenized = tokenize(input_text)
    ids = tokenized['input_ids'].tolist()[0]
    subwords = unmasker.tokenizer.convert_ids_to_tokens(ids)

    if masking_level == 'Tokens':
        tokens = str(input_text).split()
        mask_idx =  '[MASK]'
        if mask_idx is not None:
            input_masked = ' '.join(token if i != mask_idx else '[MASK]' for i, token in enumerate(tokens))
            display_input = input_masked
    if input_masked: 
        ids = tokenized['input_ids'].tolist()[0]
        subwords = unmasker.tokenizer.convert_ids_to_tokens(ids)
        res = unmasker(input_masked,  top_k=5)
        if res:
            print(res)
#             res = [{'Prediction':r['token_str'], 'Completed Sentence':r['sequence'].replace('[SEP]', '').replace('[CLS]', ''), 'Score':r['score']} for r in res]
#             res_table = pd.DataFrame(res)
#             st.table(res_table)











[{'sequence': 'אז אתמול הלכתי', 'score': 0.1412259191274643, 'token': 2039, 'token_str': 'אז'}, {'sequence': 'רק אתמול הלכתי', 'score': 0.08410400152206421, 'token': 1996, 'token_str': 'רק'}, {'sequence': 'גם אתמול הלכתי', 'score': 0.0514700748026371, 'token': 1929, 'token_str': 'גם'}, {'sequence': 'אבל אתמול הלכתי', 'score': 0.049527280032634735, 'token': 1969, 'token_str': 'אבל'}, {'sequence': 'טוב אתמול הלכתי', 'score': 0.038065142929553986, 'token': 1998, 'token_str': 'טוב'}]
